## Naive Bayes

#####Nominal: P(lsi|class)=(lsis_in_class + 1)/(lsis_in_class+total_unique_words_in_class)  
#####Continuous: P(lsi|class)=高斯分布機率(帶入平均數與標準差)

In [304]:
#一些需要的function
import numpy as np
def gauss(x,m,s):
    e=-(x-m)**2/(2*(s**2))
    g=1*np.exp(e)/(2*s*np.pi)**0.5
    return g

def remove_punc(sentences):
    import string 
    import re
    delEStr = '['+string.punctuation + u' '+']'
    delCStr = u'''[〔△〇◎◆●✦×→°☆■□◆●○๏••‧㊣℃ˍ╴←↑→．│﹙“—▲►★《》ΟΔΧ╳（）()∫&%﹪￥#@$！?,.…><*{}\\∼〈〉¬﹄•∼■`"’“×※【】  ’'　、。，‧．·‧‧：︰十；”～‘’′→＠＃＄％︿＆＊｀＝－—─＋｜／＼╲╱╱ ╱╱？『』」「\/]'''
    s=[]
    for sentence in sentences:
        if type(sentence)==unicode:
            sentence=re.sub(delEStr,u'',sentence)
            sentence=re.sub(delCStr,u'',sentence)
            s.append(sentence.encode('utf-8'))
        else:
            sentence=re.sub(delEStr,u'',sentence.decode('utf-8'))
            sentence=re.sub(delCStr,u'',sentence)
            s.append(sentence.encode('utf-8'))
    return s  

## database version

In [311]:
# 逐行從資料庫提出文章>斷詞
import MySQLdb as mc
import MySQLdb.cursors
import traceback
import jieba
import jieba.analyse
from gensim import corpora, models, similarities
import json
import time 

conn=mc.connect(host='localhost',user='root',passwd='',db='ecodaily3',charset="utf8",cursorclass = MySQLdb.cursors.SSCursor)
cur=conn.cursor()
sql='select content from news2'
cur.execute(sql)
f=open('/Users/jojotenya/Desktop/test.txt','a')
texts=cur.fetchone()
while texts is not None:
    words = jieba.cut(remove_punc([texts[0]])[0], cut_all=False)
    sentence=[]
    for w in words:
        if len(w)==1:
            pass
        else:
            sentence.append(w)
    feed=(' '.join(sentence)).encode('utf-8')
    f.write(feed)
    f.write("\n")
    texts=cur.fetchone()

cur.close()
conn.close()
f.close()


#轉換斷詞為純數字vector，生成dictionary詞庫、詞頻統計
stoplist=[]
dictionary = corpora.Dictionary(line.split() for line in open('/Users/jojotenya/Desktop/test.txt'))
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist if stopword in dictionary.token2id]
once_ids = [tokenid for tokenid, docfreq in dictionary.dfs.iteritems() if docfreq == 1]
dictionary.filter_tokens(stop_ids + once_ids) # remove stop words and words that appear only once
dictionary.compactify() # remove gaps in id sequence after words that were removed
dictionary.save('/Users/jojotenya/Desktop/test.dict')

class MyCorpus(object):
    def __iter__(self):
        for line in open('/Users/jojotenya/Desktop/test.txt'):
            yield dictionary.doc2bow(line.split())
corpus = MyCorpus() 
corpora.MmCorpus.serialize('/Users/jojotenya/Desktop/test.mm', corpus)

'''for c in corpus_to_use:
    print json.dumps(c,ensure_ascii=False)'''

#做tfidf，將斷詞編號依照vector轉換回原本的詞，一同將詞與權重存入database
#Lsi降維，將各topic與權重存入database
#print json.dumps(dictionary.token2id, encoding="utf-8", ensure_ascii=False)
##no update：
#dictionary=corpora.Dictionary.load('/Users/jojotenya/Desktop/test.dict')
#corpus=corpora.MmCorpus('/Users/jojotenya/Desktop/test.dict')
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
t=[]
for doc in corpus_tfidf:
     t.append([[dictionary.token2id.keys()[dictionary.token2id.values().index(d[0])],d[1]] for d in doc])
        
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=10)
lsi.save('/Users/jojotenya/Desktop/test.lsi')
corpus_lsi = lsi[corpus_tfidf]
l=[]
for c in corpus_lsi:
    l.append(c)       
        
try:
    conn=mc.connect(host='localhost',user='root',passwd='',db='ecodaily3',charset="utf8",cursorclass = MySQLdb.cursors.SSCursor)
    cur=conn.cursor()
    sql='select newsID,url from news2'
    cur.execute(sql)
    row=cur.fetchone()

    conn2=mc.connect(host='localhost',user='root',passwd='',db='ecodaily3',charset="utf8")
    conn2.query('SET autocommit=TRUE;')
    cur2=conn2.cursor()
    conn3=mc.connect(host='localhost',user='root',passwd='',db='ecodaily3',charset="utf8")
    conn3.query('SET autocommit=TRUE;')
    cur3=conn3.cursor()
    #print json.dumps(t[i],ensure_ascii=False)
    i=0
    while row is not None:    
        for s in t[i]:  
            data=(int(row[0]),row[1],s[0],s[1])
            cur2.execute('insert into keywords(id,url,term,weight) values(%s,%s,%s,%s)',data)
            #conn2.commit()
        for c in l[i]: 
            data=(int(row[0]),row[1],c[0],c[1])
            cur3.execute('insert into lsi(id,url,topic,weight) values(%s,%s,%s,%s)',data)
            #conn3.commit()
        i+=1
        row=cur.fetchone()
except:
    print traceback.format_exc()

finally:
    cur.close()
    conn.close()
    cur2.close()
    conn2.close()
    cur3.close()
    conn3.close()
        

#每個分類的lsi-mu,lsi-sigma
a= time.clock()
conn=mc.connect(host='localhost',user='root',passwd='',
                db='ecodaily3',charset="utf8",cursorclass = MySQLdb.cursors.SSCursor)
cur=conn.cursor()
cur.execute('select distinct(`class`) from news2')
categories=cur.fetchall()

conn2=mc.connect(host='localhost',user='root',passwd='',
                db='ecodaily3',charset="utf8",cursorclass = MySQLdb.cursors.SSCursor)
conn2.query('SET autocommit=TRUE;')
cur2=conn2.cursor()

for i in xrange(0,len(categories)):
    cur.execute('select topic,AVG(weight),STD(weight) from lsi where id in \
                (select newsID from news2 where `class` = %s) \
                GROUP BY topic',categories[i])
    result=cur.fetchone()
    while result is not None:
        data=(categories[i],result[0],result[1],result[2])
        cur2.execute('insert into lsiStatistic(`class`,topic,mean,std) value(%s,%s,%s,%s)',data)
        #conn2.commit()
        result=cur.fetchone()

cur.close()
conn.close()
cur2.close()
conn2.close()
print time.clock()-a



0.003163


In [ ]:
###分類

In [330]:
#從資料庫提出要分類的文章的每一個lsi topic，計算其屬於不同類別的gauss機率(可設定低於一定數值以下的歸類為其他)
conn=mc.connect(host='localhost',user='root',passwd='',
                db='ecodaily3',charset="utf8",cursorclass = MySQLdb.cursors.SSCursor)
cur=conn.cursor()
categories=["國際","兩岸"]   # class which want to reclass
cur.execute('select distinct(`class`) from lsiStatistic')
lsicats=[item[0] for item in cur.fetchall()]
#print json.dumps(lsicats,ensure_ascii=False)
for i in xrange(0,len(categories)):
    cur.execute('select newsID from news2 where `class` = %s;', (categories[i],))
    ID=[item[0] for item in cur.fetchall()]
    conn2=mc.connect(host='localhost',user='root',passwd='',
                    db='ecodaily3',charset="utf8",cursorclass = MySQLdb.cursors.SSCursor)
    cur2=conn2.cursor()
    conn3=mc.connect(host='localhost',user='root',passwd='',
                    db='ecodaily3',charset="utf8",cursorclass = MySQLdb.cursors.SSCursor)
    cur3=conn3.cursor()
    conn4=mc.connect(host='localhost',user='root',passwd='',
                    db='ecodaily3',charset="utf8",cursorclass = MySQLdb.cursors.SSCursor)
    conn4.query('SET autocommit=TRUE;')
    cur4=conn4.cursor()
    for n in ID:
        try:
            print n
            cur2.execute('select weight from lsi where id=%s',(n,))
            forlsicat=[item[0] for item in cur2.fetchall()]  #lsiweight for comparing to lsiStatistic
            print forlsicat
            allgauss={}
            for cat in lsicats:        
                cur3.execute('select mean,std from lsiStatistic where `class` = %s',(cat,))
                inoneclass=cur3.fetchall()
                #print inoneclass
                ini=1
                
                for m in xrange(0,len(inoneclass)):
                    ini=ini*gauss(forlsicat[m],inoneclass[m][0],inoneclass[m][1])
                allgauss[cat]=ini
            print json.dumps(allgauss,ensure_ascii=False)
            reclass=allgauss.keys()[allgauss.values().index(max(allgauss.values()))]
            print reclass
            cur4.execute('update news2 set reclass= %s where newsid =%s',(reclass,n))
            #conn4.commit()
        except:
            pass
    cur2.close()
    conn2.close()
    cur3.close()
    conn3.close()
    cur4.close()
    conn4.close()

cur.close()
conn.close()
#判斷是何種新聞類別後更新類別欄位reclass

1
[0.204402, -0.0973626, 0.0360562, 0.193477, -0.0105561, 0.0470269, -0.102649, 0.108254, -0.0703443, 0.150452]
{"國際": 0.28451250023839852, "兩岸": 0.00014206400986198963}
國際
2
[0.0788414, -0.0526129, 0.0517639, 0.0299409, -0.0227837, 0.0614836, 0.0370242, -0.0133071, -0.0680779, 0.03482]
{"國際": 0.33547819381294847, "兩岸": 0.039647548650122551}
國際
3
[0.126212, -0.0733172, 0.0438838, 0.187321, -0.035811, 0.102258, -0.07709, 0.00521589, 0.340959, -0.148317]
{"國際": 0.096523534629825852, "兩岸": 9.9823351006609553e-08}
國際
4
[0.269757, -0.277546, 0.49531, -0.341438, -0.0954633, 0.187117, 0.0530615, -0.096581, -0.251321, 0.0381869]
{"國際": 0.00012449512800852104, "兩岸": 1.3557773167103215e-11}
國際
5
[0.110476, -0.043145, -0.0163128, 0.042011, 0.0201795, -0.123099, -0.0741318, -0.124757, 0.000934131, 0.0363255]
{"國際": 0.25001327205058205, "兩岸": 0.010219250838640273}
國際
6
[0.229269, -0.190496, 0.2897, -0.110292, -0.0572117, 0.0439155, -0.103312, -0.103326, 0.470362, 0.168118]
{"國際": 0.0049924935384357

##non-database vsrsion

In [ ]:
# 逐行從資料庫提出文章>斷詞
import MySQLdb as mc
import MySQLdb.cursors
import traceback
import jieba
import jieba.analyse
from gensim import corpora, models, similarities
import json

conn=mc.connect(host='localhost',user='root',passwd='',db='ecodaily3',charset="utf8")
cur=conn.cursor()
sql='select content from news2'
cur.execute(sql)
f=open('/Users/jojotenya/Desktop/test.txt','a')
texts=cur.fetchone()
while texts is not None:
    words = jieba.cut(remove_punc([texts[0]])[0], cut_all=False)
    sentence=[]
    for w in words:
        if len(w)==1:
            pass
        else:
            sentence.append(w)
    feed=(' '.join(sentence)).encode('utf-8')
    f.write(feed)
    f.write("\n")
    texts=cur.fetchone()

cur.close()
conn.close()
f.close()


#轉換斷詞為純數字vector，生成dictionary詞庫、詞頻統計
stoplist=[]
dictionary = corpora.Dictionary(line.split() for line in open('/Users/jojotenya/Desktop/test.txt'))
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist if stopword in dictionary.token2id]
once_ids = [tokenid for tokenid, docfreq in dictionary.dfs.iteritems() if docfreq == 1]
dictionary.filter_tokens(stop_ids + once_ids) # remove stop words and words that appear only once
dictionary.compactify() # remove gaps in id sequence after words that were removed
dictionary.save('/Users/jojotenya/Desktop/test.dict')

class MyCorpus(object):
    def __iter__(self):
        for line in open('/Users/jojotenya/Desktop/test.txt'):
            yield dictionary.doc2bow(line.split())
corpus = MyCorpus() 
corpora.MmCorpus.serialize('/Users/jojotenya/Desktop/test.mm', corpus)

'''for c in corpus_to_use:
    print json.dumps(c,ensure_ascii=False)'''

#做tfidf，將斷詞編號依照vector轉換回原本的詞，一同將詞與權重存入database
#Lsi降維，將各topic與權重存入database
#print json.dumps(dictionary.token2id, encoding="utf-8", ensure_ascii=False)
##no update：
#dictionary=corpora.Dictionary.load('/Users/jojotenya/Desktop/test.dict')
#corpus=corpora.MmCorpus('/Users/jojotenya/Desktop/test.dict')
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=10)
lsi.save('/Users/jojotenya/Desktop/test.lsi')

        


#Lsi降維，將各topic與權重存入database


#每個分類的lsi-mu

#lsi_list = select * from table_name where class == class_name

#每個分類的lsi-sigma


In [220]:
###分類

In [ ]:
#從資料庫提出要分類的文章的每一個lsi topic，計算其屬於不同類別的gauss機率(可設定低於一定數值以下的歸類為其他)
conn

#判斷是何種新聞類別後更新類別欄位reclass

In [ ]:
###分群

In [ ]:
#將同一類別新聞們分群



#之後文字雲依照分類(m個)＋分群(n個)就可以有m*n個變化